In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# import logging

# from argparse import ArgumentParser
# import mne

# from moabb.benchmark import benchmark
from moabb.benchmark_parallel import BenchmarkParallel

from pathlib import Path

cwd = Path.cwd()
pipe_dir = cwd.parent / "examples" / "pipelines_braindecode"


In [18]:
from moabb import paradigms as moabb_paradigms
moabb_paradigms.__dict__.keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__path__', '__file__', '__cached__', '__builtins__', 'base', 'cvep', 'logging', 'utils', 'FakeDataset', 'BaseParadigm', 'log', 'BaseCVEP', 'CVEP', 'FilterBankCVEP', 'FakeCVEPParadigm', 'fixed_interval_windows', 'RawToFixedIntervalEvents', 'BaseProcessing', 'BaseFixedIntervalWindowsProcessing', 'FixedIntervalWindowsProcessing', 'FilterBankFixedIntervalWindowsProcessing', 'motor_imagery', 'abc', 'BaseMotorImagery', 'SinglePass', 'FilterBank', 'LeftRightImagery', 'FilterBankLeftRightImagery', 'FilterBankMotorImagery', 'MotorImagery', 'FakeImageryParadigm', 'p300', 'RawToEventsP300', 'BaseP300', 'P300', 'FakeP300Paradigm', 'resting_state', 'welch', 'RestingStateToP300Adapter', 'ssvep', 'BaseSSVEP', 'SSVEP', 'FilterBankSSVEP', 'FakeSSVEPParadigm'])

In [26]:
from moabb.paradigms import MotorImagery

p = MotorImagery()
{d.code: d.subject_list for d in p.datasets if d.code == "BNCI2014-004"}

Choosing from all possible events


{'BNCI2014-004': [1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [28]:
[d for d in p.datasets if d.code == "BNCI2014-004"][0].subject_list

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [110]:
## check if variable is string or list
if isinstance('BNCI2014_004', str):
    print('string')

string


In [178]:
from sklearn.model_selection import (
    StratifiedKFold
)
import numpy as np

X = np.array([i for i in range(20)])
# every second sample is class 1
y = np.array([0, 1] * 10)
# firsthalf is subject 0, second half is subject 1
subjects = np.array([0] * 10 + [1] * 10)

fold_idx = 4

train_idx, test_idx = [], []
for subject in np.unique(subjects):
    sss = StratifiedKFold(
        n_splits=5, shuffle=True, random_state=42
    )
    subj_idx = np.where(subjects == subject)[0]
    y_subj = y[subj_idx]
    train_idx_subj, test_idx_subj = list(sss.split(subj_idx, y_subj))[fold_idx]
    train_idx.append(subj_idx[train_idx_subj])
    test_idx.append(subj_idx[test_idx_subj])
np.concatenate(train_idx), np.concatenate(test_idx)

(array([0, 1, 3, 4, 5, 6, 7, 8]), array([2, 9]))
(array([0, 1, 3, 4, 5, 6, 7, 8]), array([2, 9]))


(array([ 0,  1,  3,  4,  5,  6,  7,  8, 10, 11, 13, 14, 15, 16, 17, 18]),
 array([ 2,  9, 12, 19]))

In [180]:
import importlib
from pathlib import Path

def read_config(config_file):
    if config_file.suffix == ".py":
        spec = importlib.util.spec_from_file_location("custom", config_file)
        foo = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(foo)
        return foo.PIPELINE
    else:
        raise ValueError("Invalid file format. Supported formats are .yml and .py")

config_file = "/Users/ldix/Documents/Projects/moabb-generalised/pipelines/my_pipes/my_shallowNet.py"

config_file = Path(config_file)
PIPELINE = read_config(config_file)


We try to set the tensorflow seeds, but it seems that tensorflow is not installed. Please refer to `https://www.tensorflow.org/` to install if you need to use this deep learning module.


{'name': 'my_ShallowFBCSPNet',
 'paradigms': ['LeftRightImagery', 'MotorImagery'],
 'return_epochs': True,
 'pipeline': Pipeline(steps=[('resample', Resampler_Epoch(sfreq=250)),
                 ('ShallowFBCSPNet',
                  <class 'braindecode.classifier.EEGClassifier'>[uninitialized](
   module=<class 'braindecode.models.shallow_fbcsp.ShallowFBCSPNet'>,
   module__final_conv_length=auto,
 ))])}

In [213]:
PIPELINE['pipeline'].steps[1][1].device

'mps'

In [214]:
from sklearn.pipeline import make_pipeline

def create_pipeline_from_config(config):
    """Create a pipeline from a config file.

    takes a config dict as input and return the corresponding pipeline.

    If the pipeline is a Tensorflow pipeline it convert also the optimizer function and the callbacks.

    Parameters
    ----------
    config : Dict.
        Dict containing the config parameters.

    Returns
    -------
    pipeline : Pipeline
        sklearn Pipeline
    """
    components = []

    for component in config:
        # load the package
        mod = __import__(component["from"], fromlist=[component["name"]])
        # create the instance
        if "parameters" in component.keys():
            params = component["parameters"]
            if "optimizer" in component["parameters"].keys():
                for optm in component["parameters"]["optimizer"]:
                    mod_optm = __import__(name=optm["from"], fromlist=[optm["name"]])
                    params_optm = optm["parameters"]
                    instance = getattr(mod_optm, optm["name"])(**params_optm)
                    component["parameters"]["optimizer"] = instance

            if "callbacks" in component["parameters"].keys():
                cb = []
                for callbacks in component["parameters"]["callbacks"]:
                    mod_callbacks = __import__(
                        name=callbacks["from"], fromlist=[callbacks["name"]]
                    )
                    params_callbacks = callbacks["parameters"]
                    instance = getattr(mod_callbacks, callbacks["name"])(
                        **params_callbacks
                    )
                    cb.append(instance)
                component["parameters"]["callbacks"] = cb

        else:
            params = {}
        instance = getattr(mod, component["name"])(**params)
        components.append(instance)

    pipeline = make_pipeline(*components)

    return pipeline

In [10]:
import yaml

with open("nesttest.yaml", "r") as f:
    config = yaml.safe_load(f)
config

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'val_f1_score'},
 'parameters': {'dataset': {'parameters': {'ds1': {'values': [1, 2, 3, 4]},
    'ds2': {'value': [1, 2, 3]}}}}}

In [15]:
config['parameters']['dataset']['parameters']

{'ds1': {'values': [1, 2, 3, 4]}, 'ds2': {'value': [1, 2, 3]}}

In [23]:
import wandb

sweep_configuration = {
    "method": "grid",
    "name": "nest-test",
    "metric": {"goal": "minimize", "name": "loss"},
    "parameters": {
        "eval": {"value": "subject"},
        # "datasets" : {"value": ["123", "456"]},
        "runs": {
            "values": [
                {"dataset" : "123", "subject_list": [1, 2, 3]},
                {"dataset" : "456", "subject_list": [4, 5, 6]},
            ]
        },
    }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="moabb-testing")

Create sweep with ID: zut1nu79
Sweep URL: https://wandb.ai/brain-lab/moabb-testing/sweeps/zut1nu79


In [24]:
import wandb

def sweep_test():
    run = wandb.init()

    print("\n\nCONFIGS")
    print(run.config)
    
    return run

sweep_id = "zut1nu79"
wandb.agent(sweep_id=sweep_id, function=sweep_test, project="moabb-testing")

wandb: Agent Starting Run: deh3ys54 with config:
wandb: 	eval: subject
wandb: 	subjects: {'dataset': '123', 'subject_list': [1, 2, 3]}




CONFIGS
{'eval': 'subject', 'subjects': {'dataset': '123', 'subject_list': [1, 2, 3]}}


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: mjvfizb0 with config:
wandb: 	eval: subject
wandb: 	subjects: {'dataset': '456', 'subject_list': [4, 5, 6]}




CONFIGS
{'eval': 'subject', 'subjects': {'dataset': '456', 'subject_list': [4, 5, 6]}}


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [27]:
wandb.run is None


True

In [8]:
import importlib

config_file = "/Users/ldix/Documents/Projects/moabb-generalised/pipelines/my_pipes/my_shallowNet.py"

if config_file.endswith(".py"):
    spec = importlib.util.spec_from_file_location("custom", config_file)
    foo = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(foo)
    foo.PIPELINE

We try to set the tensorflow seeds, but it seems that tensorflow is not installed. Please refer to `https://www.tensorflow.org/` to install if you need to use this deep learning module.


In [9]:
from pathlib import Path

[f.name for f in Path.cwd().glob("*.py")]

['bp_test.py']

In [10]:
from moabb import paradigms as moabb_paradigms


<module 'moabb.paradigms' from '/Users/ldix/Documents/Projects/moabb-generalised/moabb/paradigms/__init__.py'>

In [5]:
run = wandb.init()

print(run.config)
print(run.id)
print(run.dir)
print(run.name)
print(run.path)

{'datasets': 'BNCI2014-004', 'evaluation': 'CrossSession', 'paradigm': 'MotorImagery', 'pipeline': 'braindecode_EEGInception'}
d6clgxdy
/Users/ldix/Documents/Projects/moabb-generalised/wandb/run-20241114_091037-d6clgxdy/files
eager-sweep-36
brain-lab/moabb-testing/d6clgxdy


In [21]:
wandb.run.settings

<Settings {'_Settings__frozen': True, '_Settings__initialized': True, '_Settings__modification_order': ('_service_wait', '_stats_sample_rate_seconds', '_stats_samples_to_average', '_stats_sampling_interval', 'anonymous', 'api_key', 'base_url', 'console', 'job_source', 'mode', 'project', 'run_id', 'start_method', '_aws_lambda', 'program', '_code_path_local', '_colab', '_disable_machine_info', '_disable_meta', '_disable_stats', '_network_buffer', '_flow_control_disabled', '_flow_control_custom', '_ipython', '_jupyter', '_kaggle', '_noop', '_notebook', 'disabled', '_offline', '_shared', '_stats_neuron_monitor_config_path', 'run_mode', '_start_datetime', 'timespec', 'root_dir', 'wandb_dir', 'tmp_dir', '_tmp_code_dir', '_windows', 'colab_url', 'is_local', 'deployment', 'disable_code', 'disable_git', 'disable_job_creation', 'files_dir', '_proxies', 'http_proxy', 'https_proxy', 'log_dir', 'log_internal', 'log_symlink_internal', 'log_symlink_user', 'log_user', 'project_url', 'resume_fname', 'r